# Objaverse Downloading Script

Objaverse-XL is a Universe of 10M+ 3D Objects.
It is hosted on 🤗[Hugging Face](https://huggingface.co/datasets/allenai/objaverse-xl) and includes a [Python API on GitHub](https://github.com/allenai/objaverse-xl).

It's a bit difficult to get a dataset which is compatible for manipulation. There are a few subsets of Objaverse with additional annotations, such a Spock.

See Spock [Readme](https://github.com/allenai/spoc-robot-training) and [notebook example](https://github.com/allenai/spoc-robot-training/blob/main/how_to_use_data.ipynb)


In [12]:
# !export OBJAVERSE_PATH="/data/lmbraid19/argusm/datasets/spok/"
# !python -m objathor.dataset.download_annotations --version 2023_07_28 --path $OBJAVERSE_PATH
# !python -m objathor.dataset.download_assets --version 2023_07_28 --path $OBJAVERSE_PATH
!pip install objaverse --upgrade --quiet
!pip install compress_json

In [13]:
import objaverse
objaverse.__version__

'0.1.7'

# Spock Annotations
The rest of the cells are from the old tutorial

In [14]:
import json
from pathlib import Path
from itertools import islice
import pprint 
import numpy as np
import compress_json
import objaverse
from IPython.display import Image, display

OBJAVERSE_DIR = Path("/home/argusm/.objaverse/hf-objaverse-v1/")

#SPOCK_ANNOTATIONS_PATH = "/data/lmbraid19/argusm/datasets/spok/2023_07_28/annotations.json.gz"
#annotations_spock = compress_json.load(SPOCK_ANNOTATIONS_PATH)

SPOCK_ANNOTATIONS_PATH = "/data/lmbraid19/argusm/datasets/spok/r2_dev/annotations.json"
SPOCK_MODLES = "/data/lmbraid19/argusm/datasets/spok/r2_dev/assets"

with open(SPOCK_ANNOTATIONS_PATH, "rb") as f_obj:
    annotations_spock = json.load(f_obj)

pp = pprint.PrettyPrinter(indent=1)

In [15]:
print("annotated examples", len(annotations_spock))
annotations_spock_full = annotations_spock
annotations_spock = dict(islice(annotations_spock.items(), 5000))

annotated examples 625252


# Filtering Objaverse Objects
Objaverse has a ton of assets, so let's filter them a bit for sanity:
1. using existing annotation: `CanPickup`

In [ ]:
can_pickups = []

for i, (k,v) in enumerate(iter(annotations_spock.items())):
    try:
        can_pickup = v["thor_metadata"]["assetMetadata"]["primaryProperty"] == "CanPickup"
    except:
        can_pickup = False
    can_pickups.append(can_pickup)
    
print("can pickup:", np.mean(can_pickups)*100,"[%]")

In [8]:
print_metadata = False
if print_metadata:
    # print the metadata
    #uid = '9d17119cc0f041e39b2a11211a677366'
    uid = 'ff6c46b1e8f847ecadd5c95805f415c6'
    pp.pprint(annotations_spock_full[uid])
    ann = objaverse.load_annotations([uid])
    pp.pprint(ann[uid])

In [ ]:
import trimesh

glb_files = list((OBJAVERSE_DIR / "glbs").rglob("*.glb"))
print(f"Objaverse models found {len(glb_files)} models in {OBJAVERSE_DIR}")
objaverse_files = dict([(x.stem, x) for x in glb_files])

def get_glb_filename(uid):
    return objaverse_files[uid]

def get_glb_trimesh(uid):
    scene = trimesh.load(get_glb_filename(uid))
    return scene
    
def get_extents_file_m(uid):
    scene = trimesh.load(get_glb_filename(uid))
    geometries = list(scene.geometry.values())
    if len(geometries) > 1:
        print(f"Warning: {len(geometries)} geometries in file for {uid}")
    geometry = geometries[0]
    #print(geometry.extents)
    #print(geometry.bounding_box.extents)
    #print(geometry.bounding_box_oriented.extents)

    extents = geometry.extents
    return extents

#uid = "412ed49af0644f30bae822d29afbb066"
#print(get_extents_file_m(uid))
#mesh = get_glb_trimesh(uid)


## View Example Images with Annotations

In [ ]:
good_uids = [
'412ed49af0644f30bae822d29afbb066',
'93128128f8f848d8bd261f6c1f763a53',
'b5c9d06f19be4c92a1708515f6655573',
'584ce7acb3384c36bf252fde72063a56',
'2796ec7a4a324d9e988d95d88bb6f1e2',
'005a246f8c304e77b27cf11cd53ff4ed',
'013176cfbc8145a0b10c3191ac265e8b',
'00bfa4e5862d4d4b89f9bcf06d2a19e4',
'088c1883e07e4946956488171e3a06bf',
'0364ab96f338493c972248102b462aa4',
'ff6c46b1e8f847ecadd5c95805f415c6']

OBJAVERSE_SCALES_GUESS = {
    'b5c9d06f19be4c92a1708515f6655573':.02,
    '412ed49af0644f30bae822d29afbb066':.001,
    '088c1883e07e4946956488171e3a06bf':.1,
    '93128128f8f848d8bd261f6c1f763a53':.005
}


def find_index_with_largest_size_above_min(thubnail_image_data, min_size=448):
    valid_entries = [
        (index, entry['size']) 
        for index, entry in enumerate(thubnail_image_data) 
        if max(entry['width'], entry['height']) >= min_size
    ]
    if not valid_entries:
        return 0 # No valid entries found
    return min(valid_entries, key=lambda x: x[1])[0]


def pprint_extents(extents):
    extents = np.array(extents)*100
    return " ".join([f"{d:.1f}" for d in extents]) + " cm"

def print_spock_metadata(uid, metadata, extents_thor):
    print("category", metadata["category"])
    print("descrip", metadata["description"])
    #print("descr-auto", metadata["description_auto"])
    print("scale_thor", metadata["scale"], "extents", pprint_extents(extents_thor), "\t(apply scale_thor to extents_thor)")

def get_extents_meta_m(anno_spock):
    bbox = anno_spock['thor_metadata']['assetMetadata']['boundingBox']
    extents = {
        'x': bbox['max']['x'] - bbox['min']['x'],
        'y': bbox['max']['y'] - bbox['min']['y'],
        'z': bbox['max']['z'] - bbox['min']['z']
    }
    extents = [extents[d] for d in list('xyz')]
    return extents


filter_pass_count = 0
show_image = True
for i, uid in enumerate(annotations_spock.keys()):
#for i, uid in enumerate(good_uids):
    try:
        anno_s = annotations_spock_full[uid]
    except:
        continue

    try:
        extents_thor = get_extents_meta_m(anno_s)
    except KeyError:
        continue
    
    if min(extents_thor) < 2/100 or max(extents_thor) > 50/100:
        continue  # filter object too large to grasp

    if show_image:
        print_spock_metadata(uid, anno_s, extents_thor)
        ann = objaverse.load_annotations([uid])
        i = find_index_with_largest_size_above_min(ann[uid]['thumbnails']['images'])
        url = ann[uid]['thumbnails']['images'][i]['url']
        display(Image(url=url, width=512))
    print()

    filter_pass_count += 1
    #results = objaverse.load_objects([key])
    if i > 100:
        break

print("pass rate", filter_pass_count, "/", i)

## Compute ManiSkill Scale

Find which scales we have to give ManiSkill for objaverse objects so that they are loaded ok.
The default THOR metadata bounding box needs to be combined with the trimesh bounding box in order to find the scale.

In [ ]:
good_uids = [
'412ed49af0644f30bae822d29afbb066',
'93128128f8f848d8bd261f6c1f763a53',
'b5c9d06f19be4c92a1708515f6655573',
'584ce7acb3384c36bf252fde72063a56',
'2796ec7a4a324d9e988d95d88bb6f1e2',
'005a246f8c304e77b27cf11cd53ff4ed',
'013176cfbc8145a0b10c3191ac265e8b',
'00bfa4e5862d4d4b89f9bcf06d2a19e4',
'088c1883e07e4946956488171e3a06bf',
'0364ab96f338493c972248102b462aa4',
'ff6c46b1e8f847ecadd5c95805f415c6']

OBJAVERSE_SCALES_GUESS = {
    'b5c9d06f19be4c92a1708515f6655573':.02,
    '412ed49af0644f30bae822d29afbb066':.001,
    '088c1883e07e4946956488171e3a06bf':.1,
    '93128128f8f848d8bd261f6c1f763a53':.005
}


def pprint_extents(extents):
    extents = np.array(extents)*100
    return " ".join([f"{d:.1f}" for d in extents]) + " cm"

def print_spock_metadata(uid, metadata, extents_thor):
    print("category", metadata["category"])
    #print("descrip", metadata["description"])
    #print("descr-auto", metadata["description_auto"])
    print("scale_thor", metadata["scale"], "extents", pprint_extents(extents_thor), "\t(apply scale_thor to extents_thor)")

def get_extents_meta_m(anno_spock):
    bbox = anno_spock['thor_metadata']['assetMetadata']['boundingBox']
    extents = {
        'x': bbox['max']['x'] - bbox['min']['x'],
        'y': bbox['max']['y'] - bbox['min']['y'],
        'z': bbox['max']['z'] - bbox['min']['z']
    }
    extents = [extents[d] for d in list('xyz')]
    return extents

def guess_scale(extents_thor, extents_file):
    min_t = min(extents_thor)
    min_f = min(extents_file)
    return min_t/min_f

filter_pass_count = 0
show_image = False
#for i, key in enumerate(annotations_spock.keys()):
for i, uid in enumerate(good_uids):
    try:
        anno_s = annotations_spock_full[uid]
    except:
        continue

    try:
        extents_thor = get_extents_meta_m(anno_s)
    except KeyError:
        continue
    
    if min(extents_thor) < 2/100 or max(extents_thor) > 50/100:
        continue  # filter object too large to grasp

    print("uid", uid)
    extents_file = get_extents_file_m(uid)
    
    print_spock_metadata(uid, anno_s, extents_thor)
    scale_thor = anno_s["scale"]
    scale_pred = guess_scale(extents_thor, extents_file)
    print("                extents", pprint_extents(extents_file*scale_thor), "\t(apply scale_thor to extents_file)")
    print("                extents", pprint_extents(extents_file*scale_pred), "\t(apply scale_pred to extents_file)")
    print(f"scale_pred {scale_pred:.5f}")
    if uid in OBJAVERSE_SCALES_GUESS:
        print(f"scale_gues {OBJAVERSE_SCALES_GUESS[uid]:.5f}")

    if show_image:
        ann = objaverse.load_annotations([uid])
        url = ann[uid]['thumbnails']['images'][0]['url']
        display(Image(url=url, width=512))
    print()

    filter_pass_count += 1
    #results = objaverse.load_objects([key])
    if i > 100:
        break

print("pass rate", filter_pass_count, "/", i)

## Filter Objects by Droid Text Annotations

Make use of droid text annotations to find which objects are most frequent. 

In [ ]:
with open("droid_objects.txt") as f_obs
  text = f_obs.read()
  
items = []
for i, x in enumerate(text.split("\n")):
    item = x.split(" ")[3]
    item = item.replace("\t","")
    items.append(item)
    if item.endswith("s"):
        items.append(item[:-1])
print(len(items))

In [ ]:
from collections import Counter

# Initialize counters for categories and items
category_counter = Counter()
item_counter = Counter(items)

# Iterate through annotations and count categories and items
for uid, anno_s in annotations_spock_full.items():
    category = anno_s["category"]
    category_counter[category] += 1


# Print the unique categories that are also in the items list
categories_set = set(category_counter.keys())

item_intersection = categories_set.intersection(set(items))
for x in item_intersection:
    print(x.ljust(14), str(category_counter[x]).ljust(10), item_counter[x])

# Print frequency counts
#print("Category Frequency:", category_counter)
#print("Item Frequency:", item_counter)

## Dowload Object: Objaverse

In [ ]:
import objaverse
collection = good_uids  # list of UIDs
results = objaverse.load_objects(collection)

## Download Objects: R2-dev

Look at the pre-release r2-dev objects.

In [ ]:
import requests
from tqdm.notebook import tqdm

source = "r2_dev"
if source == "objaverse":
    res = objaverse.load_objects(good_uids)
    print(res)
elif source == "r2_dev":
    for uid in tqdm(good_uids):
        url = f"https://***REMOVED***.r2.dev/assets/{uid}.tar"
        fn = f"{SPOCK_MODLES}/{uid}.tar"
        try:
            response = requests.get(url, stream=True)  # Use streaming for large files
            response.raise_for_status()  # Raise an exception for HTTP errors
        except requests.exceptions.HTTPError:
            continue
        
        with open(fn, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):  # Write in chunks
                file.write(chunk)

### Load MSGPack Files

Try to figure out what is happening here, see if we want to load the msgpack or the original.

In [ ]:
import msgpack
print(SPOCK_MODLES)

uid = "ff6c46b1e8f847ecadd5c95805f415c6"
#uid = "412ed49af0644f30bae822d29afbb066"

msg_path = f"{SPOCK_MODLES}/{uid}/{uid}.msgpack"
with open(msg_path, "rb") as data_file:
    byte_data = data_file.read()
data_loaded = msgpack.unpackb(byte_data)
for key in data_loaded:
    sample = ""
    if type(data_loaded[key]) == list:
        sample = (len(data_loaded[key]),data_loaded[key][0])
    if isinstance(data_loaded[key], (bool, str,dict)):
        sample = data_loaded[key]

    print(key, type(data_loaded[key]), sample)

In [ ]:

print(get_glb_filename(uid))
mesh = get_glb_trimesh(uid)
tmp = list(mesh.geometry.values())[0]
print("vertices", tmp.vertices.shape)
print("triangls", tmp.triangles.shape)
print("normals", tmp.face_normals.shape)
print("material", tmp.visual.material)
print("uv", tmp.visual.uv.shape)


In [ ]:
#!pip install pygltflib
from pygltflib import GLTF2
glb_filename = get_glb_filename(uid)
glb = GLTF2().load(glb_filename)  # load method auto detects based on extension
#print(glb.asset)
#glb.bufferViews
#glb.images

# Download Images (VLM Questioning)

Download the pre-rendered images of the objects from the hosting website. 

In [ ]:
from pathlib import Path
import requests
from tqdm import tqdm_notebook as tqdm

OBJAVERSE_THUMBNAIL_PATH = Path('/data/lmbraid19/argusm/datasets/spok/thumbnails')

def find_index_with_largest_size_above_min(thubnail_image_data, min_size=448):
    valid_entries = [
        (index, entry['size']) 
        for index, entry in enumerate(thubnail_image_data) 
        if max(entry['width'], entry['height']) >= min_size
    ]
    if not valid_entries:
        return 0 # No valid entries found
    return min(valid_entries, key=lambda x: x[1])[0]


for i, (uid, value) in tqdm(enumerate(annotations_spock.items()),total=len(annotations_spock)):
    obj_anno = annotations_spock[uid]
    if "thor_metadata" not in obj_anno:
        continue
        
    try:
        anno_s = annotations_spock_full[uid]
    except:
        continue

    try:
        extents_thor = get_extents_meta_m(anno_s)
    except KeyError:
        continue
    
    if min(extents_thor) < 2/100 or max(extents_thor) > 50/100:
        continue  # filter object too large to grasp
        
    ann = objaverse.load_annotations([uid])
    thumbnail_image_data = ann[uid]['thumbnails']['images']
    i = find_index_with_largest_size_above_min(thumbnail_image_data)
    url = thumbnail_image_data[i]['url']
    fn = OBJAVERSE_THUMBNAIL_PATH / f"{uid}{Path(url).suffix}"
    if fn.is_file():
        continue

    try:
        response = requests.get(url, stream=True)  # Use streaming for large files
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.HTTPError:
        continue
    
    with open(fn, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):  # Write in chunks
            file.write(chunk)
    


# [Old-Stuff] Render in Simulation

In [ ]:
%load_ext autoreload
%autoreload 2
import gymnasium as gym
import mani_skill.examples.clevr_env  # do import to register env, not used otherwise
from mani_skill.envs.sapien_env import BaseEnv
from mani_skill.examples.gen_dataset import Args, reset_random
import matplotlib.pyplot as plt 
    
args = Args()
args.env_id = "ClevrMove-v1"
#args.shader = "rt-fast" or "rt"
#args.seed = [42,]
reset_random(args)

vis = False
env: BaseEnv = gym.make(
    args.env_id,
    obs_mode=args.obs_mode,
    reward_mode=args.reward_mode,
    control_mode=args.control_mode,
    render_mode=args.render_mode,
    sensor_configs=dict(shader_pack=args.shader),
    human_render_camera_configs=dict(shader_pack=args.shader),
    viewer_camera_configs=dict(shader_pack=args.shader),
    num_envs=args.num_envs,
    sim_backend=args.sim_backend,
    parallel_in_single_scene=False,
    robot_uids="panda_wristcam",
    object_dataset="objaverse",
    scene_dataset="Table",
    # **args.env_kwargs
)

for i in range(10**6):
    obs, _ = env.reset(seed=args.seed[0], options=dict(reconfigure=True))
    if args.seed is not None:
        env.action_space.seed(args.seed[0])
    if vis and args.render_mode is not None:
        viewer = env.render()
        if isinstance(viewer, sapien.utils.Viewer):
            viewer.paused = args.pause
        env.render()
    else:
        env.render()
    
    # get before image
    images = env.base_env.scene.get_human_render_camera_images('render_camera')
    print(i)
    plt.imshow(images['render_camera'][0].detach().cpu().numpy())
    plt.show()
    if i > 0:
        break
    reset_random(args)

# [Old-Stuff] Load all objaverse UIDs

In [ ]:
uids = objaverse.load_uids()
len(uids), type(uids)

In [ ]:
# This takes a couple of minutes
annotations = objaverse.load_annotations(uids)

In [ ]:
import pprint 
for i, (uid, annotation) in enumerate(annotations.items()):
    pp.pprint(annotation)
    #print(annotation["description"])
    if i > 2:
        break

# [Old-Stuff] LIVS Annotation Stuff

LVIS was this subset of objaverse which hand proper annotations. I was looking at it before I found the Spock annotations.

In [ ]:
import objaverse
counts = [(k,len(v)) for k,v in annotations2.items()]

In [ ]:
# select N_categories objects
N_categories = 100
print(len(counts))
sum([v for k, v in counts])
collection = []
i = 0
for i, (k, v) in enumerate(annotations2.items()):
  if i >= N_categories:
    break
  collection.append(v[0])
print(collection)

In [ ]:
results = objaverse.load_objects(collection)

### Object loading code, LVIS names

In [ ]:
# glb_downloaded = [x.stem for x in glb_files]
# glb_downloaded_set = set(glb_downloaded)            
# anno_path = objaverse_folder / "lvis-annotations.json.gz"
# import gzip, json
# with gzip.open(anno_path, "rt", encoding="utf-8") as f:
#     lvis_annotations = json.load(f)
# mapping = {}
# for category, items in lvis_annotations.items():
#     for index, item in enumerate(items, start=1):  # Use 1-based indexing
#         if item not in glb_downloaded_set:
#             continue
#         mapping[f"{category}-{index}"] = glb_files[glb_downloaded.index(item)]
# self.objaverse_model_ids = list(mapping.keys())
# self.objaverse_files = mapping